In [38]:
import importlib
import scraper

importlib.reload(scraper)  # Forces reloading of the scraper module

from scraper import fetch_sold_listings, fetch_sold_listings_test, get_listing_details, debug_pagination

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()

# Automatically match ChromeDriver with installed Chrome version
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://google.com")  # Test if Chrome opens correctly

print("Chrome opened successfully!")
driver.quit()

Chrome opened successfully!


In [ ]:
# Run the scraper
url = "https://housesigma.com/on/sold/map/?status=sold&lat=43.715564&lon=-79.418602&zoom=10&page=1&view=list"

# debug_pagination(url)

sold_data_df = fetch_sold_listings(url, 20)

Opening HouseSigma...
Scraping: https://housesigma.com/on/sold/map/?status=sold&lat=43.715564&lon=-79.418602&zoom=10&page=1&view=list
Reloaded the url, waiting for the page to load
✅ Saved HTML: housesigma_page_1.html

Retrieving data from the page
Reading html file
  Asking Price Sold Price                                            Address  \
0     $998,000   $975,000   623 - 88 Colgate Ave , Toronto - South Riverdale   
1     $609,000   $595,000  1915 - 4725 Sheppard Ave E , Scarborough - Agi...   
2     $428,000   $428,000          503 - 220 GEORGE St , Toronto - Moss Park   
3     $495,000   $488,000  511 - 8 Pemberton Ave , North York - Newtonbro...   
4     $799,000   $750,000  703 - 575 Bloor St E , Toronto - North St. Jam...   

   Unit Type Bedrooms  
0  Condo Apt        2  
1  Condo Apt        2  
2  Condo Apt        1  
3  Condo Apt        1  
4  Condo Apt        2  
Scraping: https://housesigma.com/on/sold/map/?status=sold&lat=43.715564&lon=-79.418602&zoom=10&page=2&view=l

In [35]:
print("Dataframe shape: ", sold_data_df.shape)

print(sold_data_df.head(20))

Dataframe shape:  (100, 5)
   Asking Price  Sold Price  \
0      $998,000    $975,000   
1      $609,000    $595,000   
2      $428,000    $428,000   
3      $495,000    $488,000   
4      $799,000    $750,000   
5      $399,000    $525,000   
6      $574,999    $563,000   
7      $699,900    $663,000   
8    $1,299,000  $1,400,022   
9      $534,000    $524,500   
10     $559,000    $555,000   
11     $498,800    $470,000   
12   $1,175,000  $1,120,000   
13     $860,000    $810,000   
14   $1,269,900  $1,269,900   
15     $599,888    $585,000   
16   $1,188,000  $1,200,000   
17     $625,000    $616,000   
18   $3,138,800  $2,980,000   
19     $745,000    $733,000   

                                              Address           Unit Type  \
0    623 - 88 Colgate Ave , Toronto - South Riverdale           Condo Apt   
1   1915 - 4725 Sheppard Ave E , Scarborough - Agi...           Condo Apt   
2           503 - 220 GEORGE St , Toronto - Moss Park           Condo Apt   
3   511 - 8 P

In [40]:
prices_df = get_listing_details("housesigma_page_1.html")

print(prices_df['Address'][0])
print(prices_df)

Reading html file
623 - 88 Colgate Ave , Toronto - South Riverdale
   Asking Price  Sold Price  \
0      $998,000    $975,000   
1      $609,000    $595,000   
2      $428,000    $428,000   
3      $495,000    $488,000   
4      $799,000    $750,000   
5      $399,000    $525,000   
6      $574,999    $563,000   
7      $699,900    $663,000   
8    $1,299,000  $1,400,022   
9      $534,000    $524,500   
10     $559,000    $555,000   
11     $498,800    $470,000   
12   $1,175,000  $1,120,000   
13     $860,000    $810,000   
14   $1,269,900  $1,269,900   
15     $599,888    $585,000   
16   $1,188,000  $1,200,000   
17     $625,000    $616,000   
18   $3,138,800  $2,980,000   
19     $745,000    $733,000   

                                              Address           Unit Type  \
0    623 - 88 Colgate Ave , Toronto - South Riverdale           Condo Apt   
1   1915 - 4725 Sheppard Ave E , Scarborough - Agi...           Condo Apt   
2           503 - 220 GEORGE St , Toronto - Moss P